Take several pictures of red, blue, and green items with your phone or other
digital camera.12

– Load each image, and convert it to a tensor.

– For each image tensor, use the .mean() method to get a sense of how bright
the image is.

– Now take the mean of each channel of your images. Can you identify the red,
green, and blue items from only the channel averages?

Select a relatively large file containing Python source code.

– Build an index of all the words in the source file. (Feel free to make your
tokenization as simple or as complex as you like; we suggest starting by
replacing r"[^a-zA-Z0-9_]+" with spaces.)

– Compare your index with the one you made for Pride and Prejudice. Which is
larger?

– Create the one-hot encoding for the source code file.

– What information is lost with this encoding? How does that information
compare with what’s lost in the Pride and Prejudice encoding?

In [0]:
from skimage import io, transform
import numpy as np
import torch
import imageio

In [0]:
images = ['blue-sky.jpg','green-tree.jpg','rose.jpg']

In [0]:
batch_size = len(images)
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
new_h = 256
new_w = 256

In [19]:
i = 0
for filename in images:
  print(filename)
  img_arr = io.imread(filename)
  print(img_arr.shape)
  img = transform.resize(img_arr, (new_h, new_w))
  img_in = torch.from_numpy(img)
  out = torch.transpose(img_in, 0, 2)
  batch[i] = out
  i+=1

blue-sky.jpg
(669, 1000, 3)
green-tree.jpg
(194, 259, 3)
rose.jpg
(1500, 1500, 3)


In [20]:
batch[0]

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

# Convert image to single channel

In [0]:
batch = batch.float()
batch /= 255.0

In [0]:
# In[7]:
n_channels = batch.shape[1]
for c in range(n_channels):
 mean = torch.mean(batch[:, c])
 std = torch.std(batch[:, c])
 batch[:, c] = (batch[:, c] - mean) / std

# Print the channel values 

In [24]:
# Red channel
print(batch[:,0])
# blue channel
print(batch[:,1])
# green channel
print(batch[:,2])

tensor([[[-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         ...,
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001]],

        [[-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         ...,
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001]],

        [[ 2.4993,  2.4993,  2.4993,  ...,  2.4993, -0.4001, -0.4001],
         [ 2.4993,  2.4993,  2.4993,  ...,  2

# Pride and Prejudice example

In [0]:

with open('1342-0.txt', encoding='utf8') as f:
  text = f.read()

In [28]:
lines = text.split('\n')
print(len(lines))
line = lines[100]
line

129


'    train_loader = torch.utils.data.DataLoader('

In [29]:

letter_tensor = torch.zeros(len(line), 128)
letter_tensor.shape

torch.Size([47, 128])

In [0]:
for i, letter in enumerate(line.lower().strip()):
  letter_index = ord(letter) if ord(letter) < 128 else 0
  letter_tensor[i][letter_index] = 1

In [0]:
def clean_words(input_str):
 punctuation = "[^a-zA-Z0-9_]+" #'.,;:"!?”“_-'
 word_list = input_str.lower().replace('\n',' ').split()
 word_list = [word.strip(punctuation) for word in word_list]
 return word_list

In [32]:
words_in_line = clean_words(line)
line, words_in_line

('    train_loader = torch.utils.data.DataLoader(',
 ['train_loader', '=', 'torch.utils.data.dataloader('])

In [37]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}
len(word2index_dict), word2index_dict['target']


(262, 207)

In [36]:
word2index_dict

{'': 0,
 '"cpu")': 1,
 '"mnist_cnn.pt")': 2,
 '#': 3,
 '%': 4,
 "'__main__':": 5,
 "'pin_memory':": 6,
 '(0.3081,))': 7,
 '(data,': 8,
 '(default:': 9,
 "({:.0f}%)\\n'.format(": 10,
 '({:.0f}%)]\\tloss:': 11,
 ')),': 12,
 '*': 13,
 '**kwargs)': 14,
 ".7)')": 15,
 '/': 16,
 '/=': 17,
 '1)': 18,
 "1)')": 19,
 '1):': 20,
 '1,': 21,
 "1.0)')": 22,
 '10)': 23,
 '100.': 24,
 "1000)')": 25,
 '128)': 26,
 "14)')": 27,
 '2)': 28,
 '3,': 29,
 '32,': 30,
 "64)')": 31,
 '64,': 32,
 ':': 33,
 '=': 34,
 '==': 35,
 'batch': 36,
 'batch_idx': 37,
 'batch_idx,': 38,
 'batch_size=args.batch_size,': 39,
 'batch_size=args.test_batch_size,': 40,
 'batches': 41,
 'before': 42,
 'ccuracy:': 43,
 'class': 44,
 'correct': 45,
 'correct,': 46,
 "ction='store_true',": 47,
 'cud': 48,
 'current': 49,
 'data,': 50,
 'data.to(device),': 51,
 'datasets,': 52,
 "datasets.mnist('../data',": 53,
 'def': 54,
 'default=0.7,': 55,
 'default=1,': 56,
 'default=1.0,': 57,
 'default=10,': 58,
 'default=1000,': 59,
 'default=

In [38]:
word_tensor = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
  word_index = word2index_dict[word]
  word_tensor[i][word_index] = 1
  print('{:2} {:4} {}'.format(i, word_index, word))
print(word_tensor.shape)

 0  238 train_loader
 1   34 =
 2  232 torch.utils.data.dataloader(
torch.Size([3, 262])
